In [1]:
from clear import CLEAR1
from rslvq import RSLVQ
import pandas as pd
import numpy as np

In [2]:
b_c_data = pd.read_csv('data.csv')
b_c_data.columns
b_c_data.drop(['id', 'Unnamed: 32'], axis = 1, inplace= True)
label = b_c_data['diagnosis'].replace({'M': 0, 'B': 1})
train_data = b_c_data.drop(['diagnosis'], axis = 1)
train_data

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [3]:
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
X = scale.fit_transform(train_data)
norm_t_data = pd.DataFrame(X, columns = train_data.columns, index = train_data.index)

In [4]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
x_train, x_val, y_train, y_val = train_test_split(np.array(norm_t_data), np.array(label), test_size=0.3, random_state=42)

In [5]:
trained_model = RSLVQ(num_prototypes_per_class=1)

In [6]:
prototypes,_ = trained_model.fit(x_train, y_train)

Acc.......94.72361809045226, loss......1253.4893726786345
Acc.......95.22613065326632, loss......1902.8235464448098
Acc.......95.7286432160804, loss......2414.355716885356
Acc.......95.97989949748744, loss......2842.4865137468446
Acc.......96.23115577889448, loss......3212.0908895996236
Acc.......96.4824120603015, loss......3537.225096697108
Acc.......96.73366834170855, loss......3826.964039557016
Acc.......96.73366834170855, loss......4087.660903269577
Acc.......96.98492462311557, loss......4324.007716068604
Acc.......96.98492462311557, loss......4539.605496263423
Acc.......96.98492462311557, loss......4737.299453175045
Acc.......96.98492462311557, loss......4919.388826243068
Acc.......96.98492462311557, loss......5087.764812325877
Acc.......97.23618090452261, loss......5244.00500753589
Acc.......97.23618090452261, loss......5389.44040780794
Acc.......97.23618090452261, loss......5525.204436079263
Acc.......97.23618090452261, loss......5652.26979236612
Acc.......97.23618090452261, los

In [7]:
trained_model.evaluate(x_val, y_val)

98.24561403508771

In [8]:
lab = pd.DataFrame(np.array(label), columns = ['labels'])

In [9]:
malignant_cases = norm_t_data.loc[lab['labels'] == 0]

In [10]:
new_explainer = CLEAR1(trained_model, 100, classification_threshold = 0.4, number_of_CFEs = 200, wachter_search_max = 300)
cf = new_explainer.generate_counterfactual(norm_t_data, lab, norm_t_data.iloc[0], 0, 1, prototypes)

C:\Users\amade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TypeError: RSLVQ.proba_predict() takes 2 positional arguments but 4 were given